In [65]:
# Applicable for Monatsbericht Tourismus from 2009-01 to 2022-12 (Sheet 1.9) converted with excel2csv_Monatsbericht_Tourismus_0901_2212.py

import pandas as pd
import os

def preprocess_df(df):
    df['A'] = df['A'].astype(str)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    if df.iloc[0]['A'] != 'Bundesgebiet insgesamt':
        df = df.iloc[1:]
    if df.iloc[-1]['A'] != 'Ausland':
        df = df.iloc[:-1]
    return df

directory = 'C:\\Users\\Ronja\\Documents\\Studium\\Semester 4\\DABI2\\Projekt\\Umsatzdaten\\DESTATIS\\Output\\0201_0512\\4.2\\'
dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        df = preprocess_df(df)
        dataframes.append(df)

if len(set(len(df) for df in dataframes)) > 1:
    raise ValueError("Not all dataframes have the same length")

all_data = pd.concat(dataframes, ignore_index=True)
rename_map = {'Bundesgebiet insgesamt': 'Gesamt', 'Schleswig Holstein': 'Schleswig-Holstein'}
all_data['A'] = all_data['A'].replace(rename_map)
all_data.rename(columns={'A': 'Region', 'B': 'ankuenfte_anzahl', 'C': 'ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent', 'D': 'uebernachtungen_anzahl', 'E': 'uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent', 'F': 'durchsch_aufenthaltsdauer_tage'}, inplace=True)

short_forms = ['Gesamt', 'Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
current_prefix = None

for index, row in all_data.iterrows():
    if row['Region'] in short_forms:
        current_prefix = row['Region']
    elif current_prefix:
        all_data.at[index, 'Region'] = current_prefix + '_' + row['Region']

all_data['land'] = all_data['Region'].apply(lambda x: x.split('_')[0] if '_' in x else x)
all_data['wohnsitz'] = all_data['Region'].apply(lambda x: x.split('_')[-1] if '_' in x else x)
all_data.drop(columns=['Region'], inplace=True)

all_data = all_data[['land', 'wohnsitz'] + [col for col in all_data.columns if col not in ['land', 'wohnsitz']]]
all_data['wohnsitz'] = all_data['wohnsitz'].replace({'Deutschland': 'Inland'})
all_data['wohnsitz'] = all_data['wohnsitz'].apply(lambda x: x if x in ['Inland', 'Ausland'] else 'Gesamt')

all_data[['jahr', 'monat']] = all_data['Sequence'].astype(str).str.extract(r'(\d{4})(\d{2})')
all_data.drop(columns=['Sequence'], inplace=True)

month_map = {
    '01': 'Januar', '02': 'Februar', '03': 'März', '04': 'April', '05': 'Mai', '06': 'Juni',
    '07': 'Juli', '08': 'August', '09': 'September', '10': 'Oktober', '11': 'November', '12': 'Dezember'
}
all_data['monat'] = all_data['monat'].replace(month_map)

all_data = all_data[['land', 'wohnsitz', 'jahr', 'monat'] + [col for col in all_data.columns if col not in ['land', 'wohnsitz', 'jahr', 'monat']]]

int_columns = ['jahr', 'ankuenfte_anzahl', 'uebernachtungen_anzahl']
float_columns = ['ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent', 'uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent', 'durchsch_aufenthaltsdauer_tage']
all_data[int_columns] = all_data[int_columns].apply(pd.to_numeric, errors='coerce').fillna('.')
all_data[float_columns] = all_data[float_columns].apply(pd.to_numeric, errors='coerce').fillna('.')

all_data.to_csv('C:\\Users\\Ronja\\Documents\\Studium\\Semester 4\\DABI2\\Projekt\\Umsatzdaten\\DESTATIS\\Output\\0201_0512\\herunft_0201_0512.csv', index=False)


In [66]:
all_data.tail()

,land,wohnsitz,jahr,monat,ankuenfte_anzahl,ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent,uebernachtungen_anzahl,uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent,durchsch_aufenthaltsdauer_tage
8512,Schleswig-Holstein,Inland,2022,Dezember,15055.0,-6.6,37614.0,-13.5,2.5
8513,Schleswig-Holstein,Ausland,2022,Dezember,264.0,69.2,558.0,76.0,2.1
8514,Thüringen,Gesamt,2022,Dezember,2792.0,283.0,6194.0,240.1,2.2
8515,Thüringen,Inland,2022,Dezember,2751.0,.,6054.0,.,2.2
8516,Thüringen,Ausland,2022,Dezember,41.0,.,140.0,.,3.4


In [81]:
# Applicable for Monatsbericht Tourismus from 2006-01 to 2008-12 (Sheet 4.1) converted with excel2csv_Monatsbericht_Tourismus_0601_0812.py

import pandas as pd
import os

def preprocess_df(df):
    df['A'] = df['A'].astype(str)
    df['A'] = df['A'].apply(lambda x: x.rstrip('.'))
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    if df.iloc[0]['A'] != 'Schleswig-Holstein':
        df = df.iloc[1:]
    #if df.iloc[-1]['A'] != 'Insgesamt ……………' or 'Insgesamt':
    #    df = df.iloc[:-1]
    return df

directory = 'C:\\Users\\Ronja\\Documents\\Studium\\Semester 4\\DABI2\\Projekt\\Umsatzdaten\\DESTATIS\\Output\\0601_0812\\4.1\\'
dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        df = preprocess_df(df)
        dataframes.append(df)

if len(set(len(df) for df in dataframes)) > 1:
    raise ValueError("Not all dataframes have the same length")

all_data = pd.concat(dataframes, ignore_index=True)
rename_map = {'Schleswig Holstein': 'Schleswig-Holstein', 'Insgesamt ……………': 'Gesamt'}
all_data['A'] = all_data['A'].replace(rename_map)
all_data.rename(columns={'A': 'Region', 'B': 'ankuenfte_anzahl', 'C': 'ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent', 'D': 'uebernachtungen_anzahl', 'E': 'uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent', 'F': 'durchsch_aufenthaltsdauer_tage'}, inplace=True)

short_forms = ['Bundesgebiet', 'Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
current_prefix = None

for index, row in all_data.iterrows():
    if row['Region'] in short_forms:
        current_prefix = row['Region']
    elif current_prefix:
        all_data.at[index, 'Region'] = current_prefix + '_' + row['Region']

all_data['land'] = all_data['Region'].apply(lambda x: x.split('_')[0] if '_' in x else x)
all_data['wohnsitz'] = all_data['Region'].apply(lambda x: x.split('_')[-1] if '_' in x else x)
all_data.drop(columns=['Region'], inplace=True)

all_data = all_data[['land', 'wohnsitz'] + [col for col in all_data.columns if col not in ['land', 'wohnsitz']]]
all_data['wohnsitz'] = all_data['wohnsitz'].replace({'Deutschland': 'Inland'})
all_data['wohnsitz'] = all_data['wohnsitz'].replace({'Anderer Wohnsitz': 'Ausland'})
all_data['wohnsitz'] = all_data['wohnsitz'].replace({'Zusammen': 'Gesamt'})
all_data = all_data[all_data['wohnsitz'].isin(['Inland', 'Ausland', 'Gesamt'])]
all_data['land'] = all_data['land'].replace({'Bundesgebiet': 'Gesamt'})

all_data[['jahr', 'monat']] = all_data['Sequence'].astype(str).str.extract(r'(\d{4})(\d{2})')
all_data.drop(columns=['Sequence'], inplace=True)

month_map = {
    '01': 'Januar', '02': 'Februar', '03': 'März', '04': 'April', '05': 'Mai', '06': 'Juni',
    '07': 'Juli', '08': 'August', '09': 'September', '10': 'Oktober', '11': 'November', '12': 'Dezember'
}
all_data['monat'] = all_data['monat'].replace(month_map)

all_data = all_data[['land', 'wohnsitz', 'jahr', 'monat'] + [col for col in all_data.columns if col not in ['land', 'wohnsitz', 'jahr', 'monat']]]

int_columns = ['jahr', 'ankuenfte_anzahl', 'uebernachtungen_anzahl']
float_columns = ['ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent', 'uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent', 'durchsch_aufenthaltsdauer_tage']
all_data[int_columns] = all_data[int_columns].apply(pd.to_numeric, errors='coerce').fillna('.')
all_data[float_columns] = all_data[float_columns].apply(pd.to_numeric, errors='coerce').fillna('.')

all_data.to_csv('C:\\Users\\Ronja\\Documents\\Studium\\Semester 4\\DABI2\\Projekt\\Umsatzdaten\\DESTATIS\\Output\\0601_0812\\uebernachtungen_0601_0812.csv', index=False)


In [43]:
dataframes_lengths = {}

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        df = preprocess_df(df)
        dataframes.append(df)
        dataframes_lengths[filename] = len(df)

print(dataframes_lengths)

{'MonatsberichtTourismus_0601.csv': 68, 'MonatsberichtTourismus_0602.csv': 68, 'MonatsberichtTourismus_0603.csv': 68, 'MonatsberichtTourismus_0604.csv': 68, 'MonatsberichtTourismus_0605.csv': 68, 'MonatsberichtTourismus_0606.csv': 68, 'MonatsberichtTourismus_0608.csv': 67, 'MonatsberichtTourismus_0609.csv': 67, 'MonatsberichtTourismus_0610.csv': 67, 'MonatsberichtTourismus_0611.csv': 67, 'MonatsberichtTourismus_0612.csv': 67, 'MonatsberichtTourismus_0701.csv': 67, 'MonatsberichtTourismus_0702.csv': 67, 'MonatsberichtTourismus_0703.csv': 67, 'MonatsberichtTourismus_0704.csv': 67, 'MonatsberichtTourismus_0705.csv': 67, 'MonatsberichtTourismus_0706.csv': 67, 'MonatsberichtTourismus_0708.csv': 67, 'MonatsberichtTourismus_0709.csv': 67, 'MonatsberichtTourismus_0710.csv': 67, 'MonatsberichtTourismus_0711.csv': 67, 'MonatsberichtTourismus_0712.csv': 67, 'MonatsberichtTourismus_0801.csv': 67, 'MonatsberichtTourismus_0802.csv': 67, 'MonatsberichtTourismus_0803.csv': 67, 'MonatsberichtTourismus_

In [79]:
# Applicable for Monatsbericht Tourismus from 2002-01 to 2005-12 (Sheet 3.1) converted with excel2csv_Monatsbericht_Tourismus_0201_0512.py

import pandas as pd
import os

def preprocess_df(df):
    df['A'] = df['A'].astype(str)
    df['A'] = df['A'].apply(lambda x: x.rstrip('.'))
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    if df.iloc[0]['A'] != 'Baden-Württemberg':
        df = df.iloc[1:]
    if df.iloc[-1]['A'] != 'Insgesamt':
        df = df.iloc[:-1]
    return df

directory = 'C:\\Users\\Ronja\\Documents\\Studium\\Semester 4\\DABI2\\Projekt\\Umsatzdaten\\DESTATIS\\Output\\0201_0512\\3.1\\'
dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        df = preprocess_df(df)
        dataframes.append(df)

if len(set(len(df) for df in dataframes)) > 1:
    raise ValueError("Not all dataframes have the same length")

all_data = pd.concat(dataframes, ignore_index=True)
rename_map = {'Schleswig Holstein': 'Schleswig-Holstein', 'Insgesamt': 'Gesamt'}
all_data['A'] = all_data['A'].replace(rename_map)
all_data.rename(columns={'A': 'Region', 'B': 'ankuenfte_anzahl', 'C': 'ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent', 'D': 'uebernachtungen_anzahl', 'E': 'uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent', 'F': 'durchsch_aufenthaltsdauer_tage'}, inplace=True)

short_forms = ['Bundesgebiet', 'Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
current_prefix = None

for index, row in all_data.iterrows():
    if row['Region'] in short_forms:
        current_prefix = row['Region']
    elif current_prefix:
        all_data.at[index, 'Region'] = current_prefix + '_' + row['Region']

all_data['land'] = all_data['Region'].apply(lambda x: x.split('_')[0] if '_' in x else x)
all_data['wohnsitz'] = all_data['Region'].apply(lambda x: x.split('_')[-1] if '_' in x else x)
all_data.drop(columns=['Region'], inplace=True)

all_data = all_data[['land', 'wohnsitz'] + [col for col in all_data.columns if col not in ['land', 'wohnsitz']]]
all_data['wohnsitz'] = all_data['wohnsitz'].replace({'Bundesrep. Deutschland': 'Inland'})
all_data['wohnsitz'] = all_data['wohnsitz'].replace({'Deutschland': 'Inland'})
all_data['wohnsitz'] = all_data['wohnsitz'].replace({'Anderer Wohnsitz': 'Ausland'})
all_data['wohnsitz'] = all_data['wohnsitz'].replace({'Zusammen': 'Gesamt'})
all_data = all_data[all_data['wohnsitz'].isin(['Inland', 'Ausland', 'Gesamt'])]
all_data['land'] = all_data['land'].replace({'Bundesgebiet': 'Gesamt'})

all_data[['jahr', 'monat']] = all_data['Sequence'].astype(str).str.extract(r'(\d{4})(\d{2})')
all_data.drop(columns=['Sequence'], inplace=True)

month_map = {
    '01': 'Januar', '02': 'Februar', '03': 'März', '04': 'April', '05': 'Mai', '06': 'Juni',
    '07': 'Juli', '08': 'August', '09': 'September', '10': 'Oktober', '11': 'November', '12': 'Dezember'
}
all_data['monat'] = all_data['monat'].replace(month_map)

all_data = all_data[['land', 'wohnsitz', 'jahr', 'monat'] + [col for col in all_data.columns if col not in ['land', 'wohnsitz', 'jahr', 'monat']]]

int_columns = ['jahr', 'ankuenfte_anzahl', 'uebernachtungen_anzahl']
float_columns = ['ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent', 'uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent', 'durchsch_aufenthaltsdauer_tage']
all_data[int_columns] = all_data[int_columns].apply(pd.to_numeric, errors='coerce').fillna('.')
all_data[float_columns] = all_data[float_columns].apply(pd.to_numeric, errors='coerce').fillna('.')

all_data.to_csv('C:\\Users\\Ronja\\Documents\\Studium\\Semester 4\\DABI2\\Projekt\\Umsatzdaten\\DESTATIS\\Output\\0201_0512\\uebernachtungen_0201_0512.csv', index=False)


In [82]:
all_data

,land,wohnsitz,jahr,monat,ankuenfte_anzahl,ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent,uebernachtungen_anzahl,uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent,durchsch_aufenthaltsdauer_tage
1,Schleswig-Holstein,Inland,2006,Januar,753.0,-1.6,1781.0,-30.7,2.4
2,Schleswig-Holstein,Ausland,2006,Januar,34.0,-44.3,65.0,-57.5,1.9
3,Schleswig-Holstein,Gesamt,2006,Januar,787.0,-4.7,1846.0,-32.2,2.3
5,Hamburg,Inland,2006,Januar,.,.,.,.,.
6,Hamburg,Ausland,2006,Januar,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...
2238,Thüringen,Ausland,2008,Dezember,31.0,181.8,95.0,295.8,3.1
2239,Thüringen,Gesamt,2008,Dezember,555.0,9.5,2367.0,44.1,4.3
2241,Gesamt,Inland,2008,Dezember,51923.0,28.6,211834.0,13.1,4.1
2242,Gesamt,Ausland,2008,Dezember,7667.0,24.3,21211.0,1.9,2.8


In [65]:
dataframes_lengths = {}

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        df = preprocess_df(df)
        dataframes.append(df)
        dataframes_lengths[filename] = len(df)

print(dataframes_lengths)

{'MonatsberichtTourismus_0201.csv': 68, 'MonatsberichtTourismus_0202.csv': 68, 'MonatsberichtTourismus_0203.csv': 68, 'MonatsberichtTourismus_0204.csv': 68, 'MonatsberichtTourismus_0205.csv': 68, 'MonatsberichtTourismus_0206.csv': 68, 'MonatsberichtTourismus_0207.csv': 68, 'MonatsberichtTourismus_0208.csv': 68, 'MonatsberichtTourismus_0209.csv': 68, 'MonatsberichtTourismus_0210.csv': 68, 'MonatsberichtTourismus_0211.csv': 68, 'MonatsberichtTourismus_0212.csv': 68, 'MonatsberichtTourismus_0301.csv': 68, 'MonatsberichtTourismus_0302.csv': 68, 'MonatsberichtTourismus_0303.csv': 68, 'MonatsberichtTourismus_0304.csv': 68, 'MonatsberichtTourismus_0305.csv': 68, 'MonatsberichtTourismus_0306.csv': 68, 'MonatsberichtTourismus_0307.csv': 68, 'MonatsberichtTourismus_0308.csv': 68, 'MonatsberichtTourismus_0309.csv': 68, 'MonatsberichtTourismus_0310.csv': 68, 'MonatsberichtTourismus_0311.csv': 68, 'MonatsberichtTourismus_0312.csv': 68, 'MonatsberichtTourismus_0401.csv': 68, 'MonatsberichtTourismus_